In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, TimestampType, FloatType
import pyspark.sql.functions as F

### 1. Brands Bronze Table 

In [0]:
catalog_name = 'ecommerce'

# Define the schema for the brands data file 
brand_schema = StructType([
    StructField('brand_code',StringType(),False),
    StructField('brand_name',StringType(),True),
    StructField('category_code',StringType(),True),
])

In [0]:
raw_data_path = "/Volumes/ecommerce/source_data/raw/brands/*.csv"

df = spark.read.option('header','true').option('delimiter',',').schema(brand_schema).csv(raw_data_path)


# Add metadata column 
df = df.withColumn('_source_file', F.col('_metadata.file_path')) \
        .withColumn('ingested_at', F.current_timestamp())

display(df.limit(5))

In [0]:
# Create delta table 

df.write.mode('delta') \
    .mode('overwrite') \
    .option('mergeSchema','true') \
    .saveAsTable(f'{catalog_name}.bronze.brz_brands')

### 2. Category Bronze Table

In [0]:
# Define the schema for the category data file 

category_schema = StructType([
    StructField('category_code',StringType(),False),
    StructField('category_name',StringType(),True),
])

In [0]:
raw_data_path = "/Volumes/ecommerce/source_data/raw/category/*.csv"

df_raw = spark.read.option('header', 'true').option('delimiter', ',').schema(category_schema).csv(raw_data_path)

# Add Metadata Column 
df_raw = df_raw.withColumn('_source_file', F.col('_metadata.file_path')) \
            .withColumn('ingested_at',F.current_timestamp())

display(df_raw.limit(5))


In [0]:
# Creating delta table 

df_raw.write.mode('delta') \
    .mode('overwrite') \
    .option('mergeSchema', 'true') \
    .saveAsTable(f"{catalog_name}.bronze.brz_category")


### 3. Customer Bronze Table

In [0]:
# Define the schema for the customer data file

customer_schema = StructType([
    StructField('customer_id',StringType(),False),
    StructField('phone',StringType(),True),
    StructField('country_code',StringType(),True),
    StructField('country',StringType(),True),
    StructField('state',StringType(),True),
])

In [0]:
raw_data_path = '/Volumes/ecommerce/source_data/raw/customers/*.csv'

df = spark.read.option('header', 'true').option('delimiter',',').schema(customer_schema).csv(raw_data_path)

# Add Metadata Column 
df = df.withColumn('_source_file', F.col('_metadata.file_path')) \
        .withColumn('ingested_at', F.current_timestamp())

display(df.limit(5))

In [0]:
# Creating delta table 

df.write.mode('delta') \
    .mode('overwrite') \
    .option('mergeSchema', 'true') \
    .saveAsTable(f"{catalog_name}.bronze.brz_customers")

### 4. Date Bronze Table

In [0]:
# Define schema for the data file
date_schema = StructType([
    StructField("date", StringType(), True),           # Raw date in string format
    StructField("year", IntegerType(), True),          # Year
    StructField("day_name", StringType(), True),       # Day name (can be mixed case)
    StructField("quarter", IntegerType(), True),       # Quarter
    StructField("week_of_year", IntegerType(), True),  # Week of year (can be negative)
])

# Load data using the schema defined
raw_data_path = f"/Volumes/ecommerce/source_data/raw/date/*.csv" 

df_raw = spark.read.option("header", "true").option("delimiter", ",").schema(date_schema).csv(raw_data_path)

# Add metadata columns
df_raw = df_raw.withColumn("_ingested_at", F.current_timestamp()) \
               .withColumn("_source_file", F.col("_metadata.file_path"))


# Write raw data to the Bronze layer (catalog: ecommerce, schema: bronze, table: brz_calendar) 
df_raw.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(f"{catalog_name}.bronze.brz_calendar")               

### 5. Products Bronze Table

In [0]:
products_schema = StructType([
    StructField("product_id", StringType(), False),
    StructField("sku", StringType(), True),
    StructField("category_code", StringType(), True),
    StructField("brand_code", StringType(), True),
    StructField("color", StringType(), True),
    StructField("size", StringType(), True),
    StructField("material", StringType(), True),
    StructField("weight_grams", StringType(), True),  #datatype is string due to incoming data contain anamolies
    StructField("length_cm", StringType(), True),     #datatype is string due to incoming data contain anamolies
    StructField("width_cm", FloatType(), True),
    StructField("height_cm", FloatType(), True),
    StructField("rating_count", IntegerType(), True),
    StructField("file_name", StringType(), False),
    StructField("ingest_timestamp", TimestampType(), False)
])

# Load data using the schema defined
raw_data_path = "/Volumes/ecommerce/source_data/raw/products/*.csv"

df = spark.read.option("header", "true").option("delimiter", ",").schema(products_schema).csv(raw_data_path) \
    .withColumn("file_name", F.col("_metadata.file_path")) \
    .withColumn("ingest_timestamp", F.current_timestamp())

# Write raw data to the Bronze layer (catalog: ecommerce, schema: bronze, table: brz_products)
df.write.format("delta") \
    .mode("overwrite") \
    .option("mergeSchema", "true") \
    .saveAsTable(f"{catalog_name}.bronze.brz_products")    